In [2]:
import redis
import json

# Define connection variables
host = 'localhost'
port =  6379
password = None #'password'

# Connect to Redis
r = redis.Redis(host=host, port=port, password=password, decode_responses=True)
print('Connected to Redis')

r.flushdb()

Connected to Redis


True

# Market Trend Query

## Description of the use case
This use case involves analyzing market trends by indexing and querying time-series data and related information. By understanding market trends, financial institutes can make informed decisions and adapt their strategies accordingly.

## Redis Data Structures Used
- **Redisearch**: To index and query market trend data.
- **Hashes**: To store additional information about market metrics.


In [7]:
r.flushdb()
# Mock data for the use case
market_data = {
    'metric_1001': {'name': 'Metric A', 'category': 'Personal_Loan', 'value': 100, 'timestamp': 1622505600},
    'metric_1002': {'name': 'Metric B', 'category': 'FDs', 'value': 150, 'timestamp': 1625097600},
    'metric_1003': {'name': 'Metric C', 'category': 'Home_Loan', 'value': 200, 'timestamp': 1627776000},
}

# Use Redisearch to create an index
r.execute_command('FT.CREATE', 'marketIdx', 'ON', 'HASH', 'PREFIX', '1', 'metric_', 'SCHEMA', 'name', 'TEXT', 'category', 'TAG', 'value', 'NUMERIC', 'timestamp', 'NUMERIC')


# Use Hashes to store metric information
for metric_id, metric_info in market_data.items():
    r.hset(metric_id, mapping=metric_info)

# Function to query market trends
def query_market_trends(category):
    query = f"@category:{{{category}}}"
    return r.execute_command('FT.SEARCH', 'marketIdx', query)

# Query and print market trends for 'PL' category
pl_trends = query_market_trends('Personal_Loan')
print('Market trends for Personal Loan:', pl_trends)

# Query and print market trends for 'FDs' category
fd_trends = query_market_trends('FDs')
print('Market trends for Saving Instruments:', fd_trends)

# Query and print market trends for 'Market Share' category
hl_trends = query_market_trends('Home_Loan')
print('Market trends for Market Share:', hl_trends)

# Print the market data for each metric
for metric_id in market_data.keys():
    metric_info = r.hgetall(metric_id)
    print(f'Market data for {metric_id}: {metric_info}')


Market trends for Personal Loan: [1, 'metric_1001', ['name', 'Metric A', 'category', 'Personal_Loan', 'value', '100', 'timestamp', '1622505600']]
Market trends for Saving Instruments: [1, 'metric_1002', ['name', 'Metric B', 'category', 'FDs', 'value', '150', 'timestamp', '1625097600']]
Market trends for Market Share: [1, 'metric_1003', ['name', 'Metric C', 'category', 'Home_Loan', 'value', '200', 'timestamp', '1627776000']]
Market data for metric_1001: {'name': 'Metric A', 'category': 'Personal_Loan', 'value': '100', 'timestamp': '1622505600'}
Market data for metric_1002: {'name': 'Metric B', 'category': 'FDs', 'value': '150', 'timestamp': '1625097600'}
Market data for metric_1003: {'name': 'Metric C', 'category': 'Home_Loan', 'value': '200', 'timestamp': '1627776000'}
